# Desafio - Modulo 2 - BootCamp Data Science - IGTI
Trabalho desenvolvida com a implenetação do uso do FrameWork PySpark com Apach Spark
Uso do método  Árvpre de decisão (Decision Tree) para identificação da possibilidade de ocorrência de derrame (stroke) em função das variáveis disponíveis no dataset stroke_data.csv 

In [4]:
# Criação do ambiente da Seção Spark e chevando a versão
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Desafio Modulo 2 - Ciência de dados - SparkML") \
        .getOrCreate()

spark.version

'3.3.1'

In [5]:
# Nesta celula deve ser feita a leitura do arquivo que contem o dataset a ser estudado
stroke_df = spark.read.csv('./stroke_data.csv', header='True', inferSchema='True')

In [6]:
stroke_df.show(5)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

### Pergunta 1
#### Quantos registros existem no arquivo?

In [7]:
stroke_df.count()

67135

### Pergunta 2
#### Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, habilite inferSchema=True. Use a função printSchema() da API de Dataframes.

In [8]:
stroke_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



### Pergunta 3
#### No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?

In [9]:
# Sofreram derrame (Stroke)
stroke_df.groupBy(['stroke']).count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



### Pergunta 4
#### A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e a seguir use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type). Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?

In [10]:
# Consulta de numero de pessoas que sofreram derrame por área de atuação profissional
stroke_df.createOrReplaceTempView('strokedf')
df1 = spark.sql('SELECT work_type, COUNT(stroke) AS total FROM strokedf WHERE stroke = 1 GROUP BY work_type')
df1.show()

+-------------+-----+
|    work_type|total|
+-------------+-----+
| Never_worked|   85|
|Self-employed|10807|
|      Private|23711|
|     children|  520|
|     Govt_job| 5164|
+-------------+-----+



### Pergunta 5
#### Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:

In [11]:
# Consulta sobre a proporção entre os generos entre os participantes do estudo
stroke_df.createOrReplaceTempView('strokedf')
df1 = spark.sql('SELECT gender, COUNT(stroke) AS total FROM strokedf GROUP BY gender')
df1.show()

+------+-----+
|gender|total|
+------+-----+
|Female|39530|
| Other|   11|
|  Male|27594|
+------+-----+



### Pergunta 6
#### Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que:

In [12]:
# Comparando os números de derrames entre os grupos de hipertensos e não-hipertensos
df2 = spark.sql('SELECT hypertension, stroke, COUNT(stroke) AS total FROM strokedf GROUP BY hypertension, stroke')
df2.show()
# proporção de strokes entre hipertensos é consideravelmente maior que entre os não hipertensos.

+------------+------+-----+
|hypertension|stroke|total|
+------------+------+-----+
|           1|     0| 2200|
|           1|     1| 8817|
|           0|     0|24648|
|           0|     1|31470|
+------------+------+-----+



### Pergunta 7
#### Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?

In [13]:
# Consulta com totalização de derrame por idade. E ordenação pela totalização e mostrando o item do topo da lista.
df1 = spark.sql('SELECT age, COUNT(stroke) AS total FROM strokedf WHERE stroke=1 GROUP BY age ORDER BY total DESC')
df1.show(1)

+----+-----+
| age|total|
+----+-----+
|79.0| 2916|
+----+-----+
only showing top 1 row



### Pergunta 8
#### Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.

In [14]:
stroke_df.filter((stroke_df.age > 50) & (stroke_df.stroke == 1)).count()

28938

### Pergunta 9
#### Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.

In [15]:
# Média de glicose entre os que sofreram derrame e não sofreram derrame.
df3 = spark.sql('SELECT stroke, AVG(avg_glucose_level) AS media FROM strokedf GROUP BY stroke')
df3.show()

+------+------------------+
|stroke|             media|
+------+------------------+
|     1|119.95307046938272|
|     0|103.60273130214506|
+------+------------------+



### Pergunta 10
#### Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?

In [16]:
df4 = spark.sql('SELECT stroke, AVG(bmi) AS media FROM strokedf GROUP BY stroke')
df4.show()

+------+------------------+
|stroke|             media|
+------+------------------+
|     1|29.942490629729495|
|     0|27.989678933253657|
+------+------------------+



### Pergunta 11
#### Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas: idade, BMI, hipertensão, doença do coração, nível médio de glicose. Use o conteúdo da segunda aula interativa para criar e avaliar o modelo
#### Qual a acurácia de um modelo construído?

Desenvolvimento da modelagem usado algorítimo de Árvore de Decisão adicionando com as variáveis definidas acima.

In [17]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['age', 'hypertension','heart_disease','avg_glucose_level', 'bmi'], outputCol='features')

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

In [19]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, classifier])

In [20]:
# dividir de forma aleatória os grupos de dados para criar o modelo e para testar o modelo criado
train_data, test_data = stroke_df.randomSplit([0.7, 0.3])

In [21]:
train_data.show(6)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  9|Female|41.0|           0|            0|         Yes|     Govt_job|         Rural|            64.06| 33.8|         smokes|     1|
| 11|Female|25.0|           0|            0|          No|     

In [22]:
# Treinar o modelo como os dados de treinamento, usando o pipiline criado anteriormente.
predictStrokeModel = pipeline.fit(train_data)
# Nesta fase poderá haver algum erro no processo de modelagem em função de haver valores nulos.
# Na tabela mostrada anteriormente podemos perceber que há valores nulos na coluna idade e isso deve ser corrigido.

In [23]:
predictions = predictStrokeModel.transform(test_data)
predictions.select('age', 'bmi', 'hypertension','heart_disease','avg_glucose_level', 'bmi','stroke','prediction').show(10)

+----+----+------------+-------------+-----------------+----+------+----------+
| age| bmi|hypertension|heart_disease|avg_glucose_level| bmi|stroke|prediction|
+----+----+------------+-------------+-----------------+----+------+----------+
|58.0|33.7|           1|            0|           154.24|33.7|     0|       1.0|
|82.0|33.2|           0|            0|            59.32|33.2|     1|       1.0|
|82.0|24.0|           0|            0|            234.5|24.0|     0|       1.0|
|33.0|29.9|           0|            0|           193.42|29.9|     0|       1.0|
|37.0|36.9|           0|            0|            156.7|36.9|     1|       0.0|
|70.0|24.4|           0|            0|            76.34|24.4|     1|       1.0|
|22.0|31.9|           0|            0|            72.05|31.9|     1|       0.0|
|64.0|31.2|           1|            0|            84.49|31.2|     1|       1.0|
|23.0|19.5|           0|            0|            87.52|19.5|     0|       1.0|
|81.0|17.5|           0|            0|  

In [24]:
decisionTreeModel = predictStrokeModel.stages[1]
decisionTreeModel

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_37be2dec29cd, depth=5, numNodes=37, numClasses=2, numFeatures=5

In [25]:
list(zip(assembler.getInputCols(), decisionTreeModel.featureImportances.values))

[('age', 0.9412397272376523),
 ('hypertension', 0.021663806541208135),
 ('heart_disease', 0.023442809882109204),
 ('avg_glucose_level', 0.01365365633903037)]

In [26]:
assembler.getInputCols()

['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']

In [27]:
decisionTreeModel.featureImportances

SparseVector(5, {0: 0.9412, 1: 0.0217, 3: 0.0234, 4: 0.0137})

#### Restosta da pergunta 11 na celula seguinte

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy*100

68.35209724021122

### Pergunta 12
#### Adicione ao modelo as variáveis categóricas: gênero e status de fumante. Use o conteúdo da aula interativa para lidar com as variáveis categóricas.  A acurácia (qualidade) do modelo aumentou para:

Desenvolvimento da modelagem usado algorítimo de Árvore de Decisão adicionando mais duas variáveis categóricas.

In [29]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# indexando uma variável gender (coluna do dataset) em indice numérico
gender_indexer = StringIndexer(inputCol='gender', outputCol='GenderIndexer')
# transformando o indice numério em colunas binárias (0 ou 1) - vetore
gender_encoder = OneHotEncoder(inputCol='GenderIndexer', outputCol='GenderVector')

# indexando uma variável  smoking_status (coluna do dataset) em indice numérico
smoke_indexer = StringIndexer(inputCol='smoking_status', outputCol='SmokeIndexer')
# transformando o indice numério em colunas binárias (0 ou 1) - vetore
smoke_encoder = OneHotEncoder(inputCol='SmokeIndexer', outputCol='SmokeVector')
# isso é necessário em função da necessidade de se analisar números

# Montagem dass colunas (variáveis/features) para a modelagem
assembler = VectorAssembler(inputCols=['age', 'bmi', 'hypertension','heart_disease','avg_glucose_level','GenderVector','SmokeVector']
                            , outputCol='features')

# Montando o tipo de classificação a ser executada features => stroke
classifier = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

# Construindo o pipeline ou sequencia de processos para a modelagem
pipeline = Pipeline(stages=[gender_indexer, gender_encoder, smoke_indexer, smoke_encoder, assembler, classifier])



In [30]:
# Dividindo o data frame entre dados de treino e dados de teste 
train_data, test_data = stroke_df.randomSplit([0.7, 0.3])

In [31]:
# Criando o modelo sobre os dados de treino conforme o pipeline anterior
predictStrokeModel = pipeline.fit(train_data)

In [32]:
predictions = predictStrokeModel.transform(test_data)
predictions.select('age', 'bmi', 'hypertension','heart_disease','avg_glucose_level','gender','GenderVector','smoking_status','SmokeVector','stroke','prediction').show(15)

+----+-----+------------+-------------+-----------------+------+-------------+---------------+-------------+------+----------+
| age|  bmi|hypertension|heart_disease|avg_glucose_level|gender| GenderVector| smoking_status|  SmokeVector|stroke|prediction|
+----+-----+------------+-------------+-----------------+------+-------------+---------------+-------------+------+----------+
|18.0|12.12|           0|            0|            94.19|Female|(2,[0],[1.0])|         smokes|(2,[0],[1.0])|     1|       1.0|
|82.0| 33.2|           0|            0|            59.32|Female|(2,[0],[1.0])|         smokes|(2,[0],[1.0])|     1|       1.0|
|82.0| 24.0|           0|            0|            234.5|Female|(2,[0],[1.0])|formerly smoked|(2,[1],[1.0])|     0|       1.0|
|37.0| 36.9|           0|            0|            156.7|Female|(2,[0],[1.0])|         smokes|(2,[0],[1.0])|     1|       1.0|
|72.0| 40.3|           0|            1|           235.22|  Male|(2,[1],[1.0])|formerly smoked|(2,[1],[1.0])|   

#### Restosta da pergunta 12 na célula seguinte

In [33]:
evaluator = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy*100

83.5776397515528

### Pergunta 13
#### Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu na questão (12)?

In [34]:
# O valor referente ao stages corresponde ao estágio do pipeline que representa a fase ou estagio 'classifier'
# pipeline = Pipeline(stages=[gender_indexer, gender_encoder, smoke_indexer, smoke_encoder, assembler, classifier])

decisionTreeModel = predictStrokeModel.stages[5]
decisionTreeModel

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_009623323683, depth=5, numNodes=19, numClasses=2, numFeatures=9

In [35]:
# Monta uma lista de features (variaveis) usadas para montar o modelo e associa aos valores de importãncia de cada features
features_importaces = list(zip(assembler.getInputCols(), decisionTreeModel.featureImportances.values))
features_importaces

# É bom sempre verificar os dados separadamente para saber se não há problema de apresentação dos dados

[('age', 0.16886236235303517),
 ('bmi', 0.0010033282353134124),
 ('hypertension', 0.007421700649007434),
 ('heart_disease', 0.49443870091602005),
 ('avg_glucose_level', 0.3282739078466239)]

In [36]:
# Apresenta os valores não-zero do vetor
decisionTreeModel.featureImportances.values

array([0.16886236, 0.00100333, 0.0074217 , 0.4944387 , 0.32827391])

In [37]:
assembler.getInputCols()

['age',
 'bmi',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'GenderVector',
 'SmokeVector']

### Pergunta 14
#### Qual a profundidade da árvore de decisão da questão (12)?

In [38]:
decisionTreeModel.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_009623323683, depth=5, numNodes=19, numClasses=2, numFeatures=9\n  If (feature 7 in {0.0})\n   If (feature 8 in {0.0})\n    Predict: 0.0\n   Else (feature 8 not in {0.0})\n    If (feature 0 <= 56.5)\n     Predict: 0.0\n    Else (feature 0 > 56.5)\n     If (feature 0 <= 74.5)\n      If (feature 4 <= 72.685)\n       Predict: 0.0\n      Else (feature 4 > 72.685)\n       Predict: 1.0\n     Else (feature 0 > 74.5)\n      Predict: 1.0\n  Else (feature 7 not in {0.0})\n   If (feature 0 <= 66.5)\n    Predict: 1.0\n   Else (feature 0 > 66.5)\n    If (feature 0 <= 72.5)\n     If (feature 1 <= 20.45)\n      If (feature 1 <= 17.15)\n       Predict: 1.0\n      Else (feature 1 > 17.15)\n       Predict: 0.0\n     Else (feature 1 > 20.45)\n      Predict: 1.0\n    Else (feature 0 > 72.5)\n     Predict: 1.0\n'

In [39]:
decisionTreeModel = predictStrokeModel.stages[5]
decisionTreeModel.depth

5

### Pergunta 15
#### Quantos nodos a árvore de decisão possui?

In [40]:
decisionTreeModel.numNodes

19

In [41]:
decisionTreeModel.numFeatures

9